pyESM WORKFLOW

In [1]:
# create an instance of Interface class
# based on the model settings, this generates the model's input files to be filled
# and the blank SQL database

from src.frontend.interface import Interface

test = Interface(
    log_level='debug',
    log_format='minimal',
)

INFO | Interface | 'Interface' object initialization...
INFO | Interface.file_manager | 'FileManager' object generated.
INFO | Interface | Loading settings.
DEBUG | Interface.file_manager | File 'settings.yml' loaded.
INFO | Interface.model | 'Model' object initialization...
WARNING | Interface.file_manager | Directory 'test_case_1' already exists.
DEBUG | Interface.file_manager | Directory 'test_case_1' overwritten.
INFO | Interface.model.sql_manager | Generation of 'SQLManager' object...
INFO | Interface.model.sql_manager | 'SQLManager' object generated.
INFO | Interface.model.database | 'Database' object initialization...
DEBUG | Interface.model.sql_manager | Connection to 'database.db' opened.
DEBUG | Interface.model.sql_manager | SQLite table '_set_SCENARIOS' - created.
DEBUG | Interface.model.sql_manager | SQLite table '_set_SCENARIOS' - exported to sets.xlsx.
DEBUG | Interface.model.sql_manager | SQLite table '_set_DATETIME' - created.
DEBUG | Interface.model.sql_manager | SQLit

In [2]:
# load model sets from the xlsx files filled by the user
# (data validation in excel according to the constant file is envisaged)
test.model.database.load_sets()

DEBUG | Interface.model.sql_manager | Connection to 'database.db' opened.
DEBUG | Interface.file_manager | Excel file 'sets.xlsx' loaded.
DEBUG | Interface.model.sql_manager | SQLite table '_set_SCENARIOS' - 1 entries added.
DEBUG | Interface.model.sql_manager | SQLite table '_set_DATETIME' - 2 entries added.
DEBUG | Interface.model.sql_manager | SQLite table '_set_TECHNOLOGIES' - 4 entries added.
DEBUG | Interface.model.sql_manager | SQLite table '_set_FLOWS' - 4 entries added.
INFO | Interface.model.database | New Sets loaded in Database class and 'database.db' from 'sets.xlsx'.
DEBUG | Interface.model.sql_manager | Connection to 'database.db' closed.


In [4]:
# once sets has compiled in excel and loaded in the database, variables 
# coordinates are loaded and blank database is generated
test.model.database.generate_blank_database()

DEBUG | Interface.model.sql_manager | Connection to 'database.db' opened.
INFO | Interface.model.database | Generation of empty SQLite database...
DEBUG | Interface.model.sql_manager | SQLite table 'v' already exists.
DEBUG | Interface.model.sql_manager | SQLite 'v' - deleted.
DEBUG | Interface.model.sql_manager | SQLite table 'v' - created with foreign keys.
DEBUG | Interface.model.sql_manager | SQLite table 'v' - 18 entries added.
DEBUG | Interface.model.sql_manager | Column 'values' added to table 'v'.
DEBUG | Interface.model.sql_manager | SQLite table 'u' - created with foreign keys.
DEBUG | Interface.model.sql_manager | SQLite table 'u' - 12 entries added.
DEBUG | Interface.model.sql_manager | Column 'values' added to table 'u'.
DEBUG | Interface.model.sql_manager | SQLite table 'Y' - created with foreign keys.
DEBUG | Interface.model.sql_manager | SQLite table 'Y' - 4 entries added.
DEBUG | Interface.model.sql_manager | Column 'values' added to table 'Y'.
DEBUG | Interface.model.

In [5]:
# generate input files to filled by the user
test.model.database.generate_input_files()

DEBUG | Interface.model.sql_manager | Connection to 'database.db' opened.
INFO | Interface.model.database | Generation of input file/s for 'Database' object.
DEBUG | Interface.file_manager | Directory 'input_data' created.
DEBUG | Interface.model.sql_manager | SQLite table 'v' - exported to input_data.xlsx.
DEBUG | Interface.model.sql_manager | SQLite table 'u' - exported to input_data.xlsx.
DEBUG | Interface.model.sql_manager | SQLite table 'Y' - exported to input_data.xlsx.
INFO | Interface.model.database | Input file/s for 'Database' object generated.
DEBUG | Interface.model.sql_manager | Connection to 'database.db' closed.


In [7]:
# load input file/s data into the sql database
test.model.database.load_input_files()

DEBUG | Interface.model.sql_manager | Connection to 'database.db' opened.
INFO | Interface.model.database | Loading input file/s for 'Database' object.
DEBUG | Interface.file_manager | Excel file 'input_data.xlsx' loaded.
DEBUG | Interface.model.sql_manager | 18 rows deleted from table 'v'
DEBUG | Interface.model.sql_manager | SQLite table 'v' - 18 entries added.
DEBUG | Interface.model.sql_manager | 12 rows deleted from table 'u'
DEBUG | Interface.model.sql_manager | SQLite table 'u' - 12 entries added.
DEBUG | Interface.model.sql_manager | 4 rows deleted from table 'Y'
DEBUG | Interface.model.sql_manager | SQLite table 'Y' - 4 entries added.
INFO | Interface.model.database | Input file/s loaded into 'Database' object.
DEBUG | Interface.model.sql_manager | Connection to 'database.db' closed.


In [ ]:
# use this to start from existing database
from src.frontend.interface import Interface

test = Interface(
    log_level='debug',
    log_format='minimal',
    log_file_name='model_test.log',
)

test.warm_start_for_debug()

In [ ]:
test.model.variables_generation()
test.model.variables


In [ ]:
from src.util.util import prettify
prettify(test.model.database.coordinates)

In [ ]:
from src.util.util import prettify
prettify(test.model.database.variables_info['v'])

In [ ]:
test.model.database.sets_structure

In [ ]:
test.model.database.sets

In [ ]:
test.model.database.sqltools.open_connection()

var = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        's_Name': ['Reference'],
        'dt_Name': [2023],
        't_Name': ['Gas power plant'],
        'f_Name': ['Steel']
    }
)

filter_1 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_FLOWS',
    child_column_name='f_Name',
    parent_table_fields={
        'f_Category': ['Product flow'],
    }
)

filter_2 = test.model.database.sqltools.get_related_table_keys(
    parent_table_name='_set_DATETIME',
    child_column_name='dt_Name',
    parent_table_fields={
        'dt_Name': [2023],
    }
)

var_custom_filter = test.model.database.sqltools.filtered_table_to_dataframe(
    table_name='v',
    filters_dict={
        **filter_1,
        **filter_2,
        **{'f_Name': ['Steel']},
    }
)

test.model.database.sqltools.close_connection()

# filter_1, filter_2
var_custom_filter

In [ ]:
# delete all the information included in the model data folder, if necessary
test.model.model_dir_cleanup()